In [61]:
import pandas as pd
import numpy as np
import pickle as pkl
from tsfracdiff import FractionalDifferentiator
from data_provider.data_factory import data_provider
from sklearn.linear_model import LinearRegression

In [62]:
data = f'llamatest_BTC_Daily_fracdiffnodata.csv_voc200_seq110_patch11_pred7_stride2_itr0'
with open(f'{data}.pkl', 'rb') as file :
    res = pkl.load(file)
    preds = res ['preds'] 
    trues = res ['trues']
    inputx = res['inputx']

preds

array([[[3.1053422e+05, 2.4227747e+03, 3.5022324e+01, ...,
         1.6270406e+03, 9.9824481e+02, 1.5140532e+03],
        [3.1225984e+05, 2.3978303e+03, 3.3885845e+01, ...,
         1.5870637e+03, 9.5332373e+02, 1.4449285e+03],
        [3.3766150e+05, 2.3531333e+03, 3.4211128e+01, ...,
         1.7704153e+03, 1.0767423e+03, 1.6147106e+03],
        ...,
        [3.2070838e+05, 1.8508549e+03, 2.6465252e+01, ...,
         1.6549487e+03, 9.7108600e+02, 1.5073467e+03],
        [2.5359369e+05, 1.4996279e+03, 2.1073555e+01, ...,
         1.2067791e+03, 6.1915759e+02, 1.0683342e+03],
        [2.9011147e+05, 1.8483523e+03, 2.5412327e+01, ...,
         1.4741738e+03, 8.2731110e+02, 1.3413750e+03]],

       [[3.0149750e+05, 2.7866240e+03, 3.5572086e+01, ...,
         1.5753105e+03, 1.6264152e+03, 1.1544430e+03],
        [3.0392738e+05, 2.7132671e+03, 3.4289696e+01, ...,
         1.5203597e+03, 1.5644677e+03, 1.0924279e+03],
        [3.2980162e+05, 2.7820330e+03, 3.5104321e+01, ...,
         1.705

In [63]:
datas = f'args_BTC_Daily_fracdiffsenttest.csv.pkl'
with open(datas, 'rb') as f :
    args = pkl.load(f)
if args.freq == 0 :
    args.freq = 'h'
print("args: ", args.__dict__)
# args.seq_len = 110
# args.pred_len = 7
# args.batch_size = 1
# args.patch_size = 11
args.data_path = f'BTC_Daily_fracdiffnodata.csv'
print("args: ", args.__dict__)
test_data, test_loader = data_provider(args, 'test')

args:  {'model_id': 'BTC_Daily_fracdiffsenttest.csv_llamatest__200_15_12', 'checkpoints': './checkpoints/', 'root_path': './datasets/BTC/', 'data_path': 'BTC_Daily_fracdiffsenttest.csv', 'data': 'custom', 'features': 'M', 'freq': 'h', 'target': 'close', 'embed': 'timeF', 'percent': 100, 'seq_len': 200, 'pred_len': 15, 'label_len': 7, 'scale': 1, 'decay_fac': 0.9, 'learning_rate': 0.0001, 'batch_size': 5, 'num_workers': 10, 'train_epochs': 10, 'lradj': 'type2', 'patience': 3, 'gpt_layers': 12, 'is_gpt': 0, 'isllama': 1, 'e_layers': 3, 'd_model': 5120, 'n_heads': 8, 'd_ff': 1024, 'dropout': 0.05, 'enc_in': 7, 'c_out': 7, 'patch_size': 12, 'kernel_size': 25, 'pct': 0, 'pca': 1, 'loss_func': 'mse', 'pretrain': 1, 'freeze': 1, 'model': 'llamatest', 'stride': 2, 'max_len': -1, 'hid_dim': 16, 'tmax': 10, 'voc': 1000, 'itr': 1, 'cos': 0}
args:  {'model_id': 'BTC_Daily_fracdiffsenttest.csv_llamatest__200_15_12', 'checkpoints': './checkpoints/', 'root_path': './datasets/BTC/', 'data_path': 'BTC_

In [64]:
seq_len = 110

def get_price_data(test_data, preds : np.array, index = 2, URT = 'ADF', pred_len = 7):
    fracdiff2 = FractionalDifferentiator(maxOrderBound= 1, precision = 0.01, memoryThreshold= 0.001, unitRootTest= URT)
    with open(f'./datasets/BTC/params_diff_{URT}.pkl', 'rb') as file :
        Info = pkl.load(file)
    fracdiff2.orders = Info['orders'][3:4]
    fracdiff2.numLags = Info['numLags'][3:4]
    fracdiff2.isFitted = True
    numLags = fracdiff2.numLags[0]
    f = test_data.data_stamp
    i = f[index]-1
    df = pd.read_csv(f'datasets/BTC/BTC_Daily_sentiment_newer.csv')
    input = df.iloc[i:i+seq_len+1][['open', 'high', 'low', 'close']].values
    close_lag = input[-numLags:,-1]
    pred = fracdiff2.InverseTransform(preds[index], close_lag)[-pred_len:].to_numpy().squeeze(axis = 1)
    return pred

predictions= []
for index in range(preds.shape[0]):
    pred = get_price_data(test_data, preds[:,:,-1], index = index)
    predictions.append(pred)

predictions = np.array(predictions)
predictions = pd.DataFrame(predictions)
predictions.to_csv(f'BTC_pred.csv', index = False)

In [65]:
# Préparer les données pour la régression linéaire
X = np.arange(7).reshape(-1, 1)  # Jours 0 à 6

slopes = []
r2_scores = []
adjusted_slopes = []

for index, row in predictions.iterrows():
    y = row.values.reshape(-1, 1)
    
    # Normaliser les valeurs de prédiction
    y_mean = np.mean(y)
    y_normalized = y / y_mean
    
    # Appliquer la régression linéaire sur les valeurs normalisées
    model = LinearRegression()
    model.fit(X, y_normalized)
    
    slope = model.coef_[0][0]
    r2 = model.score(X, y_normalized)
    
    # Ajuster la pente en fonction de la valeur moyenne des prédictions
    adjusted_slope = slope * y_mean
    
    slopes.append(adjusted_slope)
    r2_scores.append(r2)

indic = pd.DataFrame()

indic['slope'] = slopes
indic['r2'] = r2_scores

# # Définir des seuils pour la sélection des dates pertinentes
slope_max_threshold = np.percentile(indic['slope'], 90)
r2_threshold = np.percentile(indic['r2'], 30)

In [66]:
errors_df = pd.read_csv('errors_with_parameters.csv')

## Add the slope and r2 columns to the errors_df
errors_df['slope'] = indic['slope']

errors_df 

,RMSE,SMAPE,Sentiment,Volume,Vol,RSI_14,EMA_50,slope
0,601.748836,1.965532,25,171050.746672,5235.648281,28.071550,22909.497678,688.159157
1,1546.120821,5.196791,27,277293.710928,5588.265836,16.749731,22804.857078,329.631244
2,2665.155356,9.672708,28,163905.297979,5677.580770,15.729553,22696.339535,266.549967
3,1899.316092,6.967819,28,140191.317471,5702.460537,14.928791,22573.174014,331.096854
4,2652.905511,8.048760,24,211736.029355,5289.286125,23.910121,22483.474023,891.944054
...,...,...,...,...,...,...,...,...
448,1153.029383,1.198229,70,49565.373782,3811.516606,57.153661,32662.900171,143.137183
449,3942.065385,4.839108,63,40180.691551,3686.273172,60.771505,32817.862640,377.582176
450,3097.844829,3.900234,69,17702.878625,3407.977758,58.865452,32964.959911,259.850763
451,2491.473416,3.158578,66,22495.209788,3338.971560,62.691639,33137.452929,13.371112


In [67]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculer la matrice de corrélation
correlation_matrix = errors_df.corr()

# Afficher la heatmap de corrélation
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Heatmap de corrélation')
plt.show()


In [68]:
import statsmodels.api as sm

# Régression avec termes non linéaires
X = errors_df[['Sentiment','Volume', 'Vol', 'RSI_14', 'EMA_50', 'slope']]
y = errors_df['RMSE']
# Ajouter une constante pour l'interception
X = sm.add_constant(X)

# Modèle de régression linéaire multiple
model = sm.OLS(y, X).fit()

# Résumé du modèle
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   RMSE   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     40.91
Date:                Sun, 01 Sep 2024   Prob (F-statistic):           1.11e-39
Time:                        23:24:05   Log-Likelihood:                -3878.5
No. Observations:                 453   AIC:                             7771.
Df Residuals:                     446   BIC:                             7800.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1531.8113    632.238     -2.423      0.0

In [69]:
# Filtrer les prédictions selon les seuils définis
relevant_dates = indic[((indic['slope'] > slope_max_threshold)) & (indic['r2'] > r2_threshold)]

print(f"slope_max = {slope_max_threshold}, r2_max = {r2_threshold}")
# indic.describe()

# # Afficher les dates pertinentes
relevant_dates

# listes des index des dates pertinentes
relevant_dates_index = relevant_dates.index
relevant_dates_index

print(relevant_dates_index)

# Base date
base_date = pd.to_datetime('2023-03-13')

# Create the relevant dates column
relevant_dates['dates'] = base_date + pd.to_timedelta(relevant_dates.index, unit='D')

relevant_dates = relevant_dates[['dates']]
relevant_dates.to_csv('relevant_dates.csv', index = False)
print(relevant_dates.size)

slope_max = 684.5936137181216, r2_max = 0.9047790910757991
Index([  0,   4, 224, 247, 267, 301, 338, 350, 351, 352, 353, 354, 355, 357,
       359, 363, 364, 366, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380,
       381, 382, 383, 384, 385, 398, 401, 403, 408, 410, 427, 439, 441],
      dtype='int64')
41
